### Пункты 1-4

In [90]:
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier 
import xgboost
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

In [14]:
inp = open('SMSSpamCollection.txt', 'r')
cl_list = []
txt_list = []
for line in inp:
    cl, txt = line.split('\t')
    cl_list.append(1 if cl == 'spam' else 0)
    txt_list.append(txt)

In [18]:
countVectorizer = CountVectorizer()
X = countVectorizer.fit_transform(txt_list)

### Пункт 5

In [31]:
np.mean(cross_val_score(LogisticRegression(), X, cl_list, cv=10, scoring='f1'))

0.93334852685794145

### Пункт 6

In [25]:
samples = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
"FreeMsg: Txt: claim your reward of 3 hours talk time",
"Have you visited the last lecture on physics?",
"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
"Only 99$"]

In [26]:
log_regr = LogisticRegression()
log_regr.fit(X, cl_list)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
log_regr.predict(countVectorizer.transform(samples))

array([1, 1, 0, 0, 0])

### Пункт 7

In [36]:
def evaluate(method):
    results = []
    for ngram_range in [(2, 2), (3, 3), (1, 3)]:
        countVectorizer = CountVectorizer(ngram_range=ngram_range)
        X = countVectorizer.fit_transform(txt_list)
        results.append(np.mean(cross_val_score(method, X, cl_list, cv=10, scoring='f1')))
    return map(lambda x: round(x, 2), results)

In [37]:
evaluate(LogisticRegression())

[0.82, 0.73, 0.93]

### Пункт 8

In [43]:
evaluate(MultinomialNB())

[0.65, 0.38, 0.89]

### Пункт 9

In [49]:
tfidfVectorizer = TfidfVectorizer()
X = tfidfVectorizer.fit_transform(txt_list)

In [50]:
log_regr = LogisticRegression()
log_regr.fit(X, cl_list)
np.mean(cross_val_score(log_regr, X, cl_list, cv=10, scoring='f1'))

0.85285995541724557

качество понизилось

### Пункт 10

In [52]:
def try_method(ngram_range, method):
    countVectorizer = CountVectorizer(ngram_range=ngram_range)
    X = countVectorizer.fit_transform(txt_list)
    return np.mean(cross_val_score(method, X, cl_list, cv=10, scoring='f1'))

In [59]:
try_method((1, 1), RandomForestClassifier(n_estimators=1000))

0.91071777042176605

In [65]:
#countVectorizer = CountVectorizer()
#X = countVectorizer.fit_transform(txt_list)
#print np.mean(cross_val_score(GradientBoostingClassifier(), X.toarray(), cl_list, cv=10, scoring='f1'))

In [92]:
countVectorizer = CountVectorizer()
X = countVectorizer.fit_transform(txt_list).toarray()
xtrain, xval, ytrain, yval = train_test_split(X, cl_list, random_state=0, test_size=.2)

Xdatatrain = xgboost.DMatrix(data = xtrain, label = ytrain)
Xdatatest = xgboost.DMatrix(data = xval, label = yval)
watchlist = [(Xdatatrain, 'train'), (Xdatatest, 'eval')]            

param = {}
param['booster'] = 'gbtree'
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['silent'] = 0

numround = 150

plst = list(param.items())

In [93]:
bst = xgboost.train(plst, Xdatatrain, numround, evals = watchlist,  early_stopping_rounds=50, verbose_eval = 10)
score = log_loss(yval, bst.predict(Xdatatest))
print "final score = {}".format(score)

[0]	train-logloss:0.47604	eval-logloss:0.481482
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.092936	eval-logloss:0.121982
[20]	train-logloss:0.058126	eval-logloss:0.099573
[30]	train-logloss:0.045623	eval-logloss:0.088486
[40]	train-logloss:0.038643	eval-logloss:0.082606
[50]	train-logloss:0.033865	eval-logloss:0.078594
[60]	train-logloss:0.029991	eval-logloss:0.075876
[70]	train-logloss:0.027444	eval-logloss:0.076249
[80]	train-logloss:0.025269	eval-logloss:0.075936
[90]	train-logloss:0.023679	eval-logloss:0.076031
[100]	train-logloss:0.022602	eval-logloss:0.076136
[110]	train-logloss:0.02153	eval-logloss:0.075852
[120]	train-logloss:0.020388	eval-logloss:0.075817
[130]	train-logloss:0.019238	eval-logloss:0.076924
[140]	train-logloss:0.018539	eval-logloss:0.076816
final score = 0.0781032709326


In [94]:
print "final xgboost (no tuning) score = {}".format(1-score)

final xgboost (no tuning) score = 0.921896729067


### Пункт 11

Дано мало примеров, мы для них получаем количество фич, существенно превышающее кол-во примеров, поэтому сложным методам не хватает информации для качественного обучения; простейшая модель логистической регрессии дает лучший результат из всего испробованного. Если генерировать больше фич (2 и 3 н-граммы), то прироста это не дает опять же из-за малого количества сэпмлов.